<a href="https://colab.research.google.com/github/mpuglin/AlgorithmicTrading/blob/master/VWAP2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread pandas numpy google-auth -q

**VWAP STRATEGY**

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
from datetime import datetime
import numpy as np

# Authenticate and authorize Google Sheets
creds, _ = default()
gc = gspread.authorize(creds)

# Initialize global variables to track strategy performance
strategy_results = []

def get_clean_financial_data(sheet):
    """Download and clean financial data from a single Google Sheet worksheet"""
    # Get all data from the sheet
    rows = sheet.get_all_values()

    if len(rows) < 2:
        print(f"Skipping sheet {sheet.title}: Not enough rows.")
        return None

    headers = [h.strip() for h in rows[0]]
    required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

    missing_columns = [col for col in required_columns if col not in headers]
    if missing_columns:
        print(f"Skipping sheet {sheet.title}: Missing required columns - {missing_columns}")
        return None

    col_indices = {col: headers.index(col) for col in required_columns}

    # Prepare data dictionary
    data = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    for row in rows[1:]:
        if len(row) < len(headers):
            continue  # Skip incomplete rows

        try:
            # Parse date
            date_str = row[col_indices['Date']]
            parsed_date = parse_date(date_str)
            if not parsed_date:
                continue

            data['Date'].append(parsed_date)
            data['Open'].append(float(row[col_indices['Open']]))
            data['High'].append(float(row[col_indices['High']]))
            data['Low'].append(float(row[col_indices['Low']]))
            data['Close'].append(float(row[col_indices['Close']]))
            data['Volume'].append(int(float(row[col_indices['Volume']])))
        except ValueError as e:
            print(f"Skipping row in sheet {sheet.title} due to error: {e}")
            continue

    if not data['Date']:
        print(f"Skipping sheet {sheet.title}: No valid data found.")
        return None

    # Create DataFrame and clean
    df = pd.DataFrame(data)
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    df = df.ffill()  # Forward fill missing values

    return df

def parse_date(date_str):
    """Parse date string with multiple possible formats"""
    formats = [
        '%m/%d/%Y %H:%M:%S',  # 01/02/2024 00:00:00
        '%m/%d/%Y',            # 01/02/2024
        '%Y-%m-%d %H:%M:%S',   # 2024-01-02 00:00:00
        '%Y-%m-%d',            # 2024-01-02
        '%d-%m-%Y %H:%M:%S',   # 02-01-2024 00:00:00
        '%d-%m-%Y'             # 02-01-2024
    ]

    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    print(f"Warning: Could not parse date string: {date_str}")
    return None

def calculate_vwap(df):
    """Calculate Volume Weighted Average Price (VWAP) without lookahead bias"""
    # Calculate typical price (average of high, low, and close)
    df['Typical Price'] = (df['High'] + df['Low'] + df['Close']) / 3

    # Initialize cumulative columns
    df['Cumulative Volume'] = 0.0
    df['Cumulative PV'] = 0.0
    df['VWAP'] = 0.0

    # Compute cumulative values iteratively (row-by-row)
    cum_volume = 0.0
    cum_pv = 0.0

    for i in range(len(df)):
        cum_volume += df['Volume'].iloc[i]
        cum_pv += df['Typical Price'].iloc[i] * df['Volume'].iloc[i]

        df.loc[df.index[i], 'Cumulative Volume'] = cum_volume
        df.loc[df.index[i], 'Cumulative PV'] = cum_pv
        df.loc[df.index[i], 'VWAP'] = cum_pv / cum_volume if cum_volume > 0 else 0.0

    return df

def trading_strategy(data):
    """Pure VWAP trading strategy without lookahead bias"""
    data['Signal'] = 0  # Initialize signals

    # Calculate VWAP standard deviation bands using expanding window
    data['VWAP_std'] = data['VWAP'].expanding(min_periods=20).std()
    data['Upper Band'] = data['VWAP'] + data['VWAP_std']
    data['Lower Band'] = data['VWAP'] - data['VWAP_std']

    # Generate signals (no lookahead)
    for i in range(1, len(data)):
        # Buy when price approaches lower band (within 0.5%)
        if (data['Close'].iloc[i] - data['Lower Band'].iloc[i]) / data['Lower Band'].iloc[i] <= 0.005:
            data.loc[data.index[i], 'Signal'] = 1

        # Sell when price approaches upper band (within 0.5%)
        elif (data['Upper Band'].iloc[i] - data['Close'].iloc[i]) / data['Upper Band'].iloc[i] <= 0.005:
            data.loc[data.index[i], 'Signal'] = -1

    return data


def backtest_strategy(data, initial_capital=10000, transaction_fee=0.001):
    """Backtest the VWAP trading strategy"""
    position = 0  # 0 = No position, 1 = Long
    entry_price = 0
    portfolio_value = [initial_capital]  # Track portfolio value over time
    first_buy_occurred = False  # Flag to track the first buy signal
    transaction_log = []  # Log to store transaction details

    for i in range(len(data)):
        if data['Signal'].iloc[i] == 1 and position == 0:  # Buy signal
            if not first_buy_occurred:
                first_buy_occurred = True  # Mark the first buy signal
            position = 1
            entry_price = data['Close'].iloc[i]
            fee = round(portfolio_value[-1] * transaction_fee, 2)  # Calculate fee
            portfolio_value[-1] = round(portfolio_value[-1] - fee, 2)  # Deduct fee
            shares = round((portfolio_value[-1] * (1 - transaction_fee)) / entry_price, 2)  # Calculate number of shares
            transaction_log.append({
                'Date': data.index[i],
                'Action': 'Buy',
                'Price': round(entry_price, 2),
                'Shares': shares,
                'Fee': fee,
                'Portfolio Value After': portfolio_value[-1]
            })

        elif data['Signal'].iloc[i] == -1 and position == 1:  # Sell signal
            if first_buy_occurred:  # Only sell if the first buy has occurred
                position = 0
                exit_price = data['Close'].iloc[i]
                shares = transaction_log[-1]['Shares']  # Get shares from last buy transaction
                portfolio_value[-1] = round(shares * exit_price, 2)  # Update capital
                fee = round(portfolio_value[-1] * transaction_fee, 2)  # Calculate fee
                portfolio_value[-1] = round(portfolio_value[-1] - fee, 2)  # Deduct fee
                transaction_log.append({
                    'Date': data.index[i],
                    'Action': 'Sell',
                    'Price': round(exit_price, 2),
                    'Shares': shares,
                    'Fee': fee,
                    'Portfolio Value After': portfolio_value[-1]
                })

        portfolio_value.append(portfolio_value[-1])  # Update portfolio value for the next iteration

    # Remove the last duplicate value
    portfolio_value = portfolio_value[:-1]

    # Add portfolio value to the dataframe
    data['Portfolio Value'] = portfolio_value

    return data, pd.DataFrame(transaction_log)

def print_strategy_summary():
    """Print summary statistics of the strategy performance across all stocks"""
    if not strategy_results:
        print("No strategy results to summarize.")
        return

    df = pd.DataFrame(strategy_results)

    # Calculate win/loss metrics
    df['Result'] = np.where(df['Return'] >= 0, 'Win', 'Loss')
    win_rate = len(df[df['Result'] == 'Win']) / len(df) * 100

    # Calculate overall metrics
    avg_return = df['Return'].mean()
    best_stock = df.loc[df['Return'].idxmax()]
    worst_stock = df.loc[df['Return'].idxmin()]

    print("\n" + "="*60)
    print("VWAP STRATEGY SUMMARY REPORT".center(60))
    print("="*60)

    print(f"\nTotal Stocks Analyzed: {len(df)}")
    print(f"Winning Stocks: {len(df[df['Result'] == 'Win'])} ({win_rate:.1f}%)")
    print(f"Losing Stocks: {len(df[df['Result'] == 'Loss'])} ({100-win_rate:.1f}%)")
    print(f"\nAverage Return: {avg_return:.2f}%")

    print("\nBest Performing Stock:")
    print(f"  {best_stock['Stock']}: {best_stock['Return']:.2f}% (${best_stock['Final Value']:,.2f})")

    print("\nWorst Performing Stock:")
    print(f"  {worst_stock['Stock']}: {worst_stock['Return']:.2f}% (${worst_stock['Final Value']:,.2f})")

    print("\n" + "="*60)

# Main execution
if __name__ == "__main__":
    # Parameters
    INITIAL_CAPITAL = 10000
    TRANSACTION_FEE = 0.001  # 0.1%

    # Open the Google Sheet
    spreadsheet = gc.open('US_Stocks')

    # Process each worksheet
    for sheet in spreadsheet.worksheets():
        try:
            print(f"\n{'='*50}")
            print(f"Processing stock: {sheet.title}")
            print(f"{'='*50}")

            # Get and process data
            data = get_clean_financial_data(sheet)
            if data is None:
                continue

            data = calculate_vwap(data)
            data = trading_strategy(data)
            results, transactions = backtest_strategy(data, INITIAL_CAPITAL, TRANSACTION_FEE)

            # Calculate results
            final_value = results['Portfolio Value'].iloc[-1]
            returns_pct = ((final_value - INITIAL_CAPITAL) / INITIAL_CAPITAL * 100)

            # Store results for summary
            strategy_results.append({
                'Stock': sheet.title,
                'Final Value': final_value,
                'Return': returns_pct,
                'Trades': len(transactions),
                'First Trade': transactions['Date'].min() if len(transactions) > 0 else None,
                'Last Trade': transactions['Date'].max() if len(transactions) > 0 else None
            })

            # Print results
            print(f"\nFinal Portfolio Value: ${final_value:,.2f}")
            print(f"Return: {returns_pct:.2f}%")
            print(f"\nNumber of trades: {len(transactions)}")

            if len(transactions) > 0:
                print("\nFirst few transactions:")
                print(transactions.head())

        except Exception as e:
            print(f"Error processing sheet {sheet.title}: {e}")
            continue

    # Print final summary after all stocks are processed
    print_strategy_summary()


Processing stock: AAPL

Final Portfolio Value: $10,016.32
Return: 0.16%

Number of trades: 5

First few transactions:
        Date Action   Price  Shares    Fee  Portfolio Value After
0 2024-01-31    Buy  184.40   54.12  10.00                9990.00
1 2024-02-05   Sell  187.68   54.12  10.16               10147.08
2 2024-02-13    Buy  185.04   54.73  10.15               10136.93
3 2024-05-03   Sell  183.38   54.73  10.04               10026.35
4 2025-04-04    Buy  188.38   53.12  10.03               10016.32

Processing stock: MSFT

Final Portfolio Value: $12,456.00
Return: 24.56%

Number of trades: 8

First few transactions:
        Date Action   Price  Shares    Fee  Portfolio Value After
0 2024-04-30    Buy  389.33   25.63  10.00                9990.00
1 2024-05-14   Sell  416.56   25.63  10.68               10665.75
2 2024-08-05    Buy  395.15   26.94  10.67               10655.08
3 2024-09-12   Sell  427.00   26.94  11.50               11491.88
4 2024-10-31    Buy  406.35   28.22

**COMPARE VWAP WITH BUY AND HOLD STRATEGY**  

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
from datetime import datetime
import numpy as np

# Authenticate and authorize Google Sheets
creds, _ = default()
gc = gspread.authorize(creds)

# Initialize global variables to track strategy performance
strategy_results = []

def get_clean_financial_data(sheet):
    """Download and clean financial data from a single Google Sheet worksheet"""
    # Get all data from the sheet
    rows = sheet.get_all_values()

    if len(rows) < 2:
        print(f"Skipping sheet {sheet.title}: Not enough rows.")
        return None

    headers = [h.strip() for h in rows[0]]
    required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

    missing_columns = [col for col in required_columns if col not in headers]
    if missing_columns:
        print(f"Skipping sheet {sheet.title}: Missing required columns - {missing_columns}")
        return None

    col_indices = {col: headers.index(col) for col in required_columns}

    # Prepare data dictionary
    data = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    for row in rows[1:]:
        if len(row) < len(headers):
            continue  # Skip incomplete rows

        try:
            # Parse date
            date_str = row[col_indices['Date']]
            parsed_date = parse_date(date_str)
            if not parsed_date:
                continue

            data['Date'].append(parsed_date)
            data['Open'].append(float(row[col_indices['Open']]))
            data['High'].append(float(row[col_indices['High']]))
            data['Low'].append(float(row[col_indices['Low']]))
            data['Close'].append(float(row[col_indices['Close']]))
            data['Volume'].append(int(float(row[col_indices['Volume']])))
        except ValueError as e:
            print(f"Skipping row in sheet {sheet.title} due to error: {e}")
            continue

    if not data['Date']:
        print(f"Skipping sheet {sheet.title}: No valid data found.")
        return None

    # Create DataFrame and clean
    df = pd.DataFrame(data)
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    df = df.ffill()  # Forward fill missing values

    return df

def parse_date(date_str):
    """Parse date string with multiple possible formats"""
    formats = [
        '%m/%d/%Y %H:%M:%S',  # 01/02/2024 00:00:00
        '%m/%d/%Y',            # 01/02/2024
        '%Y-%m-%d %H:%M:%S',   # 2024-01-02 00:00:00
        '%Y-%m-%d',            # 2024-01-02
        '%d-%m-%Y %H:%M:%S',   # 02-01-2024 00:00:00
        '%d-%m-%Y'             # 02-01-2024
    ]

    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    print(f"Warning: Could not parse date string: {date_str}")
    return None

def calculate_vwap(df):
    """Calculate Volume Weighted Average Price (VWAP) without lookahead bias"""
    # Calculate typical price (average of high, low, and close)
    df['Typical Price'] = (df['High'] + df['Low'] + df['Close']) / 3

    # Initialize cumulative columns
    df['Cumulative Volume'] = 0.0
    df['Cumulative PV'] = 0.0
    df['VWAP'] = 0.0

    # Compute cumulative values iteratively (row-by-row)
    cum_volume = 0.0
    cum_pv = 0.0

    for i in range(len(df)):
        cum_volume += df['Volume'].iloc[i]
        cum_pv += df['Typical Price'].iloc[i] * df['Volume'].iloc[i]

        df.loc[df.index[i], 'Cumulative Volume'] = cum_volume
        df.loc[df.index[i], 'Cumulative PV'] = cum_pv
        df.loc[df.index[i], 'VWAP'] = cum_pv / cum_volume if cum_volume > 0 else 0.0

    return df

def trading_strategy(data):
    """Pure VWAP trading strategy without lookahead bias"""
    data['Signal'] = 0  # Initialize signals

    # Calculate VWAP standard deviation bands using expanding window
    data['VWAP_std'] = data['VWAP'].expanding(min_periods=20).std()
    data['Upper Band'] = data['VWAP'] + data['VWAP_std']
    data['Lower Band'] = data['VWAP'] - data['VWAP_std']

    # Generate signals (no lookahead)
    for i in range(1, len(data)):
        # Buy when price approaches lower band (within 0.5%)
        if (data['Close'].iloc[i] - data['Lower Band'].iloc[i]) / data['Lower Band'].iloc[i] <= 0.005:
            data.loc[data.index[i], 'Signal'] = 1

        # Sell when price approaches upper band (within 0.5%)
        elif (data['Upper Band'].iloc[i] - data['Close'].iloc[i]) / data['Upper Band'].iloc[i] <= 0.005:
            data.loc[data.index[i], 'Signal'] = -1

    return data

def buy_and_hold_strategy(data, initial_capital=10000, transaction_fee=0.001):
    """Simple buy-and-hold strategy for comparison"""
    # Buy at the first available price
    first_close = data['Close'].iloc[0]
    shares = (initial_capital * (1 - transaction_fee)) / first_close
    initial_fee = initial_capital * transaction_fee

    # Sell at the last available price
    last_close = data['Close'].iloc[-1]
    final_value = shares * last_close
    final_fee = final_value * transaction_fee
    final_value_after_fees = final_value - final_fee

    # Calculate portfolio value over time
    data['BH Portfolio Value'] = data['Close'] * shares
    data['BH Portfolio Value'] = data['BH Portfolio Value'] - (data['BH Portfolio Value'] * transaction_fee).shift(1).fillna(initial_fee)

    # Create transaction log
    transaction_log = pd.DataFrame([{
        'Date': data.index[0],
        'Action': 'Buy',
        'Price': first_close,
        'Shares': shares,
        'Fee': initial_fee,
        'Portfolio Value After': initial_capital - initial_fee
    }, {
        'Date': data.index[-1],
        'Action': 'Sell',
        'Price': last_close,
        'Shares': shares,
        'Fee': final_fee,
        'Portfolio Value After': final_value_after_fees
    }])

    return data, transaction_log

def backtest_strategy(data, initial_capital=10000, transaction_fee=0.001):
    """Backtest the VWAP trading strategy"""
    position = 0  # 0 = No position, 1 = Long
    entry_price = 0
    portfolio_value = [initial_capital]  # Track portfolio value over time
    first_buy_occurred = False  # Flag to track the first buy signal
    transaction_log = []  # Log to store transaction details

    for i in range(len(data)):
        if data['Signal'].iloc[i] == 1 and position == 0:  # Buy signal
            if not first_buy_occurred:
                first_buy_occurred = True  # Mark the first buy signal
            position = 1
            entry_price = data['Close'].iloc[i]
            fee = round(portfolio_value[-1] * transaction_fee, 2)  # Calculate fee
            portfolio_value[-1] = round(portfolio_value[-1] - fee, 2)  # Deduct fee
            shares = round((portfolio_value[-1] * (1 - transaction_fee)) / entry_price, 2)  # Calculate number of shares
            transaction_log.append({
                'Date': data.index[i],
                'Action': 'Buy',
                'Price': round(entry_price, 2),
                'Shares': shares,
                'Fee': fee,
                'Portfolio Value After': portfolio_value[-1]
            })

        elif data['Signal'].iloc[i] == -1 and position == 1:  # Sell signal
            if first_buy_occurred:  # Only sell if the first buy has occurred
                position = 0
                exit_price = data['Close'].iloc[i]
                shares = transaction_log[-1]['Shares']  # Get shares from last buy transaction
                portfolio_value[-1] = round(shares * exit_price, 2)  # Update capital
                fee = round(portfolio_value[-1] * transaction_fee, 2)  # Calculate fee
                portfolio_value[-1] = round(portfolio_value[-1] - fee, 2)  # Deduct fee
                transaction_log.append({
                    'Date': data.index[i],
                    'Action': 'Sell',
                    'Price': round(exit_price, 2),
                    'Shares': shares,
                    'Fee': fee,
                    'Portfolio Value After': portfolio_value[-1]
                })

        portfolio_value.append(portfolio_value[-1])  # Update portfolio value for the next iteration

    # Remove the last duplicate value
    portfolio_value = portfolio_value[:-1]

    # Add portfolio value to the dataframe
    data['VWAP Portfolio Value'] = portfolio_value

    return data, pd.DataFrame(transaction_log)

def print_strategy_summary():
    """Print summary statistics of the strategy performance across all stocks"""
    if not strategy_results:
        print("No strategy results to summarize.")
        return

    df = pd.DataFrame(strategy_results)

    # Calculate win/loss metrics for both strategies
    df['VWAP Result'] = np.where(df['VWAP Return'] >= 0, 'Win', 'Loss')
    df['BH Result'] = np.where(df['BH Return'] >= 0, 'Win', 'Loss')

    vwap_win_rate = len(df[df['VWAP Result'] == 'Win']) / len(df) * 100
    bh_win_rate = len(df[df['BH Result'] == 'Win']) / len(df) * 100

    # Calculate outperformance
    df['Outperformance'] = df['VWAP Return'] - df['BH Return']
    outperformance_count = len(df[df['Outperformance'] > 0])
    outperformance_pct = outperformance_count / len(df) * 100
    avg_outperformance = df['Outperformance'].mean()

    # Calculate overall metrics
    avg_vwap_return = df['VWAP Return'].mean()
    avg_bh_return = df['BH Return'].mean()

    best_vwap_stock = df.loc[df['VWAP Return'].idxmax()]
    worst_vwap_stock = df.loc[df['VWAP Return'].idxmin()]

    best_bh_stock = df.loc[df['BH Return'].idxmax()]
    worst_bh_stock = df.loc[df['BH Return'].idxmin()]

    print("\n" + "="*80)
    print("STRATEGY COMPARISON SUMMARY REPORT".center(80))
    print("="*80)

    print(f"\nTotal Stocks Analyzed: {len(df)}")

    print("\nVWAP Strategy Performance:")
    print(f"  Winning Stocks: {len(df[df['VWAP Result'] == 'Win'])} ({vwap_win_rate:.1f}%)")
    print(f"  Losing Stocks: {len(df[df['VWAP Result'] == 'Loss'])} ({100-vwap_win_rate:.1f}%)")
    print(f"  Average Return: {avg_vwap_return:.2f}%")

    print("\nBuy-and-Hold Strategy Performance:")
    print(f"  Winning Stocks: {len(df[df['BH Result'] == 'Win'])} ({bh_win_rate:.1f}%)")
    print(f"  Losing Stocks: {len(df[df['BH Result'] == 'Loss'])} ({100-bh_win_rate:.1f}%)")
    print(f"  Average Return: {avg_bh_return:.2f}%")

    print("\nStrategy Comparison:")
    print(f"  VWAP outperformed Buy-and-Hold in {outperformance_count} stocks ({outperformance_pct:.1f}%)")
    print(f"  Average outperformance: {avg_outperformance:.2f}%")

    print("\nBest Performing VWAP Stock:")
    print(f"  {best_vwap_stock['Stock']}: {best_vwap_stock['VWAP Return']:.2f}% (${best_vwap_stock['VWAP Final Value']:,.2f})")

    print("\nBest Performing Buy-and-Hold Stock:")
    print(f"  {best_bh_stock['Stock']}: {best_bh_stock['BH Return']:.2f}% (${best_bh_stock['BH Final Value']:,.2f})")

    print("\n" + "="*80)

# Main execution
if __name__ == "__main__":
    # Parameters
    INITIAL_CAPITAL = 10000
    TRANSACTION_FEE = 0.001  # 0.1%

    # Open the Google Sheet
    spreadsheet = gc.open('US_Stocks')

    # Process each worksheet
    for sheet in spreadsheet.worksheets():
        try:
            print(f"\n{'='*50}")
            print(f"Processing stock: {sheet.title}")
            print(f"{'='*50}")

            # Get and process data
            data = get_clean_financial_data(sheet)
            if data is None:
                continue

            data = calculate_vwap(data)
            data = trading_strategy(data)

            # Backtest both strategies
            vwap_results, vwap_transactions = backtest_strategy(data.copy(), INITIAL_CAPITAL, TRANSACTION_FEE)
            bh_results, bh_transactions = buy_and_hold_strategy(data.copy(), INITIAL_CAPITAL, TRANSACTION_FEE)

            # Merge results
            results = vwap_results.copy()
            results['BH Portfolio Value'] = bh_results['BH Portfolio Value']

            # Calculate results for both strategies
            vwap_final_value = results['VWAP Portfolio Value'].iloc[-1]
            vwap_returns_pct = ((vwap_final_value - INITIAL_CAPITAL) / INITIAL_CAPITAL * 100)

            bh_final_value = results['BH Portfolio Value'].iloc[-1]
            bh_returns_pct = ((bh_final_value - INITIAL_CAPITAL) / INITIAL_CAPITAL * 100)

            # Store results for summary
            strategy_results.append({
                'Stock': sheet.title,
                'VWAP Final Value': vwap_final_value,
                'VWAP Return': vwap_returns_pct,
                'VWAP Trades': len(vwap_transactions),
                'BH Final Value': bh_final_value,
                'BH Return': bh_returns_pct,
                'BH Trades': len(bh_transactions),
                'First Trade': vwap_transactions['Date'].min() if len(vwap_transactions) > 0 else None,
                'Last Trade': vwap_transactions['Date'].max() if len(vwap_transactions) > 0 else None
            })

            # Print results
            print(f"\nVWAP Strategy:")
            print(f"  Final Portfolio Value: ${vwap_final_value:,.2f}")
            print(f"  Return: {vwap_returns_pct:.2f}%")
            print(f"  Number of trades: {len(vwap_transactions)}")

            print(f"\nBuy-and-Hold Strategy:")
            print(f"  Final Portfolio Value: ${bh_final_value:,.2f}")
            print(f"  Return: {bh_returns_pct:.2f}%")
            print(f"  Number of trades: {len(bh_transactions)}")

            print(f"\nComparison:")
            print(f"  Difference: ${vwap_final_value - bh_final_value:,.2f}")
            print(f"  Percentage Difference: {vwap_returns_pct - bh_returns_pct:.2f}%")

            if len(vwap_transactions) > 0:
                print("\nFirst few VWAP transactions:")
                print(vwap_transactions.head())

        except Exception as e:
            print(f"Error processing sheet {sheet.title}: {e}")
            continue

    # Print final summary after all stocks are processed
    print_strategy_summary()


Processing stock: AAPL

VWAP Strategy:
  Final Portfolio Value: $10,016.32
  Return: 0.16%
  Number of trades: 5

Buy-and-Hold Strategy:
  Final Portfolio Value: $10,667.17
  Return: 6.67%
  Number of trades: 2

Comparison:
  Difference: $-650.85
  Percentage Difference: -6.51%

First few VWAP transactions:
        Date Action   Price  Shares    Fee  Portfolio Value After
0 2024-01-31    Buy  184.40   54.12  10.00                9990.00
1 2024-02-05   Sell  187.68   54.12  10.16               10147.08
2 2024-02-13    Buy  185.04   54.73  10.15               10136.93
3 2024-05-03   Sell  183.38   54.73  10.04               10026.35
4 2025-04-04    Buy  188.38   53.12  10.03               10016.32

Processing stock: MSFT

VWAP Strategy:
  Final Portfolio Value: $12,456.00
  Return: 24.56%
  Number of trades: 8

Buy-and-Hold Strategy:
  Final Portfolio Value: $12,893.64
  Return: 28.94%
  Number of trades: 2

Comparison:
  Difference: $-437.64
  Percentage Difference: -4.38%

First few V